In [1]:
# Most of the data we downloaded before is in HTML form, which means they are mostly a mess.
# In this section, we try to build a tool to clean the data.

# Import the necessary modules
import os
from bs4 import BeautifulSoup
import re

if not os.path.exists("./ECB_PC_processed_transcripts/"):
    os.makedirs("./ECB_PC_processed_transcripts/")
for year in range(1998, 2020):
    # Change to the working directory
    os.chdir(r"C:\Users\TPhan\Downloads\ECB releases")
    # Cite the current working directory
    base_directory = os.getcwd()
    # List the file names in the folder corresponding to each year
    file_name = os.listdir(base_directory + "\\" + "ECB PC transcripts" + "\\" + str(year))
    name = [i[:-4] for i in file_name]
    # Now, open each file in the folder for each year
    for file in name:
        # Set working directory
        os.chdir(base_directory + "\\" + "ECB PC transcripts" + "\\" + str(year))
        # Import the data from .txt files.
        try:
            with open(file + ".txt", "r") as f:
                data = f.read()
        except UnicodeDecodeError:
            with open(file + ".txt", encoding="utf8") as f:
                data = f.read()
        # After opening each file, parse the HTML form
        html_doc = BeautifulSoup(data, "html.parser")
        # Extract the text
        text = html_doc.find_all("p")
        transcript = []
        for i in text:
            transcript.append(i.getText())
            
        # Now, the text looks much cleaner and is ready for further analysis
        # But, still, the header and the footer are needed to be discarded

        # Notice that before the main speech, there is a line of "Navigation Path: ..."
        # and after the transcript, we may observe the "Media contacts" line. Use these
        # strings as anchors for trimming the text.

        try:
            header = re.compile("Jump to the transcript ")
            footer = re.compile("Reproduction is permitted ")
            h = []
            f = []
            for index, i in enumerate(transcript):
                try:
                    header.search(i).span()
                    h.append(index)
                except AttributeError:
                    pass
                
            for index, i in enumerate(transcript):
                try:
                    footer.search(i).span()
                    f.append(index)
                except AttributeError:
                    pass
            # Trim the list, we will get the clean transcripts,
            # It is now ready for further analysis
            transcript = transcript[int(h[0])+1:int(f[0])]
        except IndexError:
            header = re.compile("Navigation Path: ")
            footer = re.compile("Reproduction is permitted ")
            h = []
            f = []
            for index, i in enumerate(transcript):
                try:
                    header.search(i).span()
                    h.append(index)
                except AttributeError:
                    pass
                
            for index, i in enumerate(transcript):
                try:
                    footer.search(i).span()
                    f.append(index)
                except AttributeError:
                    pass
            transcript = transcript[int(h[0])+1:int(f[0])]
        
        # Though, the text is still splited in elements, we are gonna stick them
        new_text = transcript[0]
        # Each element now is splited by a new line. So we can think of them as paragraphs
        for i in transcript[1:]:
            new_text = new_text + "\n" + i
        # Write it in the new .txt file
        # Change to the opened folder
        os.chdir(r"C:\Users\TPhan\Downloads\ECB releases")
        if not os.path.exists("./ECB_PC_processed_transcripts/" + str(year)):
            os.makedirs("./ECB_PC_processed_transcripts/" + str(year))
        # Save the new_text as a processed .txt file
        cwd = os.getcwd()
        os.chdir(cwd + "/ECB_PC_processed_transcripts/" + str(year))
        try:
            tran = open(file + ".txt", "w")
            tran.write(new_text)
        except UnicodeEncodeError:
            tran = open(file + ".txt", "w", encoding="utf-8")
            tran.write(new_text)